In [15]:
from copy import deepcopy
import clingo

from sudoku_func import test_inputs, check_solution, pretty_repr

In [2]:
test_input = test_inputs[0]
sudoku = test_input.sudoku
k = test_input.k

In [144]:
program = """
var(0..9).
% check_same_cell(I, I') :- var(I), var(I'), I==I'.
cell(I, J):- var(I),var(J).
% cell(I, J):- var(I),var(J), not check_same_cell(I,J).

% knights constraint
% decides if cell (I, J) is attacked by a knights move by cell (I', J')
% knight_attacked(I, J, I', J') :- cell(I, J), cell(I', J'), I' - I >= -2, I' - I <= 2, I!=I', J' - J >= -2, J' - J <= 2, J!=J', I' - I != J'-J, I' - I != J - J'.

row_diff(I, I') :- var(I), var(I'), (I - I') <= 2, (I - I') >= -2.
col_diff(I, I') :- var(I), var(I'), (I - I') <= 2, (I - I') >= -2.

knight_attacked(I, J, I', J') :- cell(I, J), cell(I', J'), I' - I >= -2, I' - I <= 2, I!=I', J' - J >= -2, J' - J <= 2, J!=J', I' - I != J'-J, I' - I != J - J'.
% knight_attacked(I, J, I', J') :- cell(I, J), cell(I', J'), row_diff(I, I'), col_diff(J, J').


#show knight_attacked/4.
"""

In [145]:
# program = """
# var(1..9).
# check_same_cell(I, I') :- var(I), var(I'), I==I'.
# cell(I, J):- var(I),var(J).

# % knights constraint
# % decides if cell (I, J) is attacked by a knights move by cell (I', J')
# knight_attacked_1(I, J, I', J') :- cell(I, J), cell(I', J'), I' == I + 2, J' = J + 1.
# knight_attacked_2(I, J, I', J') :- cell(I, J), cell(I', J'), I' == I - 2, J' = J + 1.
# my_property :- cell(I, J), cell(I', J'), knight_attacked_1(I, J, I', J').
# my_property :- cell(I, J), cell(I', J'), knight_attacked_2(I, J, I', J').
# :- not my_property


# %#show knight_attacked/4.
# """

In [146]:
def obtain_answer_set(program):
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    control.configuration.solve.models = 0
    for model in control.solve(yield_=True):
        sorted_model = [[n.number for n in atom.arguments] for atom in model.symbols(shown=True)]
        return sorted_model

In [147]:
solution = obtain_answer_set(program)

In [148]:
chess_board = [["-" for _ in range(9)] for _ in range(9)]

In [149]:
print(pretty_repr(chess_board, 3))

.-------.-------.-------.
| - - - | - - - | - - - | 
| - - - | - - - | - - - | 
| - - - | - - - | - - - | 
.-------.-------.-------.
| - - - | - - - | - - - | 
| - - - | - - - | - - - | 
| - - - | - - - | - - - | 
.-------.-------.-------.
| - - - | - - - | - - - | 
| - - - | - - - | - - - | 
| - - - | - - - | - - - | 
.-------.-------.-------.


In [150]:
pivot = 2, 2

solution_chess_board = deepcopy(chess_board)

for s in solution:
    if s[0] == pivot[0] and s[1] == pivot[1]:
        solution_chess_board[s[0]][s[1]] = "P"
        solution_chess_board[s[2]][s[3]] = "X"

print(pretty_repr(solution_chess_board, 3))

.-------.-------.-------.
| - X - | X - - | - - - | 
| X - - | - X - | - - - | 
| - - P | - - - | - - - | 
.-------.-------.-------.
| X - - | - X - | - - - | 
| - X - | X - - | - - - | 
| - - - | - - - | - - - | 
.-------.-------.-------.
| - - - | - - - | - - - | 
| - - - | - - - | - - - | 
| - - - | - - - | - - - | 
.-------.-------.-------.


In [153]:
solution

[[2, 1, 0, 0],
 [1, 2, 0, 0],
 [3, 1, 1, 0],
 [0, 2, 1, 0],
 [2, 2, 1, 0],
 [0, 1, 2, 0],
 [4, 1, 2, 0],
 [1, 2, 2, 0],
 [3, 2, 2, 0],
 [1, 1, 3, 0],
 [5, 1, 3, 0],
 [2, 2, 3, 0],
 [4, 2, 3, 0],
 [2, 1, 4, 0],
 [6, 1, 4, 0],
 [3, 2, 4, 0],
 [5, 2, 4, 0],
 [3, 1, 5, 0],
 [7, 1, 5, 0],
 [4, 2, 5, 0],
 [6, 2, 5, 0],
 [4, 1, 6, 0],
 [8, 1, 6, 0],
 [5, 2, 6, 0],
 [7, 2, 6, 0],
 [5, 1, 7, 0],
 [9, 1, 7, 0],
 [6, 2, 7, 0],
 [8, 2, 7, 0],
 [6, 1, 8, 0],
 [7, 2, 8, 0],
 [9, 2, 8, 0],
 [7, 1, 9, 0],
 [8, 2, 9, 0],
 [2, 0, 0, 1],
 [2, 2, 0, 1],
 [1, 3, 0, 1],
 [3, 0, 1, 1],
 [3, 2, 1, 1],
 [0, 3, 1, 1],
 [2, 3, 1, 1],
 [0, 0, 2, 1],
 [4, 0, 2, 1],
 [0, 2, 2, 1],
 [4, 2, 2, 1],
 [1, 3, 2, 1],
 [3, 3, 2, 1],
 [1, 0, 3, 1],
 [5, 0, 3, 1],
 [1, 2, 3, 1],
 [5, 2, 3, 1],
 [2, 3, 3, 1],
 [4, 3, 3, 1],
 [2, 0, 4, 1],
 [6, 0, 4, 1],
 [2, 2, 4, 1],
 [6, 2, 4, 1],
 [3, 3, 4, 1],
 [5, 3, 4, 1],
 [3, 0, 5, 1],
 [7, 0, 5, 1],
 [3, 2, 5, 1],
 [7, 2, 5, 1],
 [4, 3, 5, 1],
 [6, 3, 5, 1],
 [4, 0, 6, 1],
 [8, 0, 6,